In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import gzip
import pickle
file_path = '/content/drive/MyDrive/Team Interactions: SpoofProof - AudioClassification/test_dataset_feature_vectors.pkl.gz'

with gzip.open(file_path, 'rb') as f:
    feature_vecs = pickle.load(f)

In [ ]:
print(feature_vecs[0])

[[[ 0.03202868  0.05559912 -0.13610491 ... -0.15748227  0.0740476
    0.02739431]
  [-0.00051177  0.04572993 -0.10390855 ... -0.15370743  0.08055145
    0.0363284 ]
  [-0.05432767  0.01714287 -0.08007845 ... -0.18143171  0.13754745
    0.09746373]
  ...
  [-0.07185999 -0.14487886  0.00725227 ...  0.18187168 -0.02920096
   -0.18963881]
  [-0.16284624 -0.06954732 -0.13799787 ... -0.04461658  0.03304188
    0.0578848 ]
  [-0.04909662 -0.0133954  -0.12193249 ... -0.19310974  0.09140291
    0.18672557]]]


In [ ]:
import pickle

file_path = '/content/drive/MyDrive/Team Interactions: SpoofProof - AudioClassification/test_dataset.pkl'

with open(file_path, 'rb') as f:
  loaded_data = pickle.load(f)

#print(loaded_data)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Team Interactions: SpoofProof - AudioClassification/test_dataset.pkl'

In [ ]:
#NOT USING CSV, FORMAT ISSUE
import os
import pandas as pd

train_file = '/content/drive/MyDrive/Team Interactions: SpoofProof - AudioClassification/train_dataset.csv'
test_file = '/content/drive/MyDrive/Team Interactions: SpoofProof - AudioClassification/test_dataset.csv'

train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)

In [ ]:
#Resample audio to 16000HZ, can be done later
import librosa
import numpy as np

arrays = []
filepaths = loaded_data['audio']  # Assuming this contains file paths or a reference to the audio files

for i in range(len(loaded_data)):
    # Load each audio and let librosa detect the sampling rate
    y, original_sr = librosa.load(filepaths[i], sr=None)  # sr=None keeps the original rate

    # Resample the audio to 16000 Hz if needed
    resampled_audio = librosa.resample(y, orig_sr=original_sr, target_sr=16000)

    arrays.append(resampled_audio)

# Add the resampled audio back to the DataFrame
loaded_data['audio_resampled'] = arrays

# Optionally save to pickle or CSV
loaded_data.to_pickle('resampled_audio_train.pkl')

<ipython-input-25-c0a365ef97ab>:10: UserWarning: PySoundFile failed. Trying audioread instead.
  y, original_sr = librosa.load(filepaths[i], sr=None)  # sr=None keeps the original rate
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/Team Interactions: SpoofProof - AudioClassification/Authentic/Train/common_voice_en_19717598 (1).mp3'

In [ ]:
#Check audio sampling rate
import pandas as pd
import librosa
import numpy as np
import pickle

audio_sample = loaded_data['audio'][3000]  # Take the first audio sample for example
inferred_sr = librosa.get_samplerate(audio_sample)
print(inferred_sr)

48000


In [ ]:
#creating processed_features containing extracted features
import pandas as pd
import numpy as np
from transformers import AutoFeatureExtractor

# Load a feature extractor for your audio model
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

# Process each audio file to extract features
def extract_features(filepath):
    # Load the audio file (you can use librosa or other libraries)
    audio, sr = librosa.load(filepath, sr=16000)
    # Extract features using the feature extractor
    inputs = feature_extractor(audio, sampling_rate=sr, return_tensors="pt")  # Example with return tensors
    return inputs

# Create the processed_features column by applying the function to each audio file
loaded_data['processed_features'] = loaded_data['audio'].apply(extract_features)

# Now loaded_data has a column with processed features
print(loaded_data[['audio', 'processed_features']].head())


/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


                                               audio processed_features
0  /content/drive/MyDrive/Team Interactions: Spoo...     [input_values]
1  /content/drive/MyDrive/Team Interactions: Spoo...     [input_values]
2  /content/drive/MyDrive/Team Interactions: Spoo...     [input_values]
3  /content/drive/MyDrive/Team Interactions: Spoo...     [input_values]
4  /content/drive/MyDrive/Team Interactions: Spoo...     [input_values]


In [ ]:
#convert to dataset for training
import pandas as pd
from datasets import Dataset

# Assuming loaded_data is a DataFrame
dataset = Dataset.from_pandas(loaded_data[['processed_features', 'label']])  # Specify the relevant columns

# Check the contents of the Dataset
print(dataset)

ArrowInvalid: ("Could not convert {'input_values': tensor([[0.0104, 0.0104, 0.0104,  ..., 0.0104, 0.0104, 0.0104]])} with type BatchFeature: did not recognize Python value type when inferring an Arrow data type", 'Conversion failed for column processed_features with type object')

In [ ]:
#HUGGINGFACE PREPROCESSING
from transformers import AutoFeatureExtractor
import pandas as pd
from datasets import Dataset, Audio

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

#dataset = Dataset.from_pandas(loaded_data)

#sampling rate
minds = loaded_data.cast_column("number array", Audio(sampling_rate=16_000))
minds["train"][0]

#function for resampling the rate
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000, truncation=True
    )
    return inputs

encoded_minds = train_df.map(preprocess_function)

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


AttributeError: 'DataFrame' object has no attribute 'cast_column'